 Before you start using this notebook install **com.databricks.spark.xml** Maven module on your Apache Spark and then change **datalake_name** in the Python variable and file system command


In [ ]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
datalake_name = 'cadlstoren2ygnppyehpf2'

In [ ]:
%fs ls adl://cadlstoren2ygnppyehpf2.azuredatalakestore.net/

In [ ]:
%python
configs = {
  'fs.adl.oauth2.access.token.provider.type': 'CustomAccessTokenProvider',
  'fs.adl.oauth2.access.token.custom.provider': spark.conf.get('spark.databricks.passthrough.adls.tokenProviderClassName')
}
dbutils.fs.mount(
source = f'adl://{datalake_name}.azuredatalakestore.net/',
mount_point = '/mnt/datalake',
extra_configs = configs)

In [ ]:
path = f'adl://{datalake_name}.azuredatalakestore.net/Posts.xml'
posts_output_path = f'adl://{datalake_name}.azuredatalakestore.net/posts.parquet'

most_popular_questions_with_unique_tag_parquet_path = f'adl://{datalake_name}.azuredatalakestore.net/most_popular_questions_with_unique_tag.parquet'
most_popular_answers_with_unique_tag_parquet_path = f'adl://{datalake_name}.azuredatalakestore.net/most_popular_answers_with_unique_tag.parquet'
most_popular_accepted_answers_with_unique_tag_parquet_path = f'adl://{datalake_name}.azuredatalakestore.net/most_popular_accepted_answers_with_unique_tag.parquet'


lowest_popular_questions_with_unique_tag_parquet_path = f'adl://{datalake_name}.azuredatalakestore.net/lowest_popular_questions_with_unique_tag.parquet'
lowest_popular_answers_with_unique_tag_parquet_path = f'adl://{datalake_name}.azuredatalakestore.net/lowest_popular_answers_with_unique_tag.parquet'
lowest_popular_accepted_answers_with_unique_tag_parquet_path = f'adl://{datalake_name}.azuredatalakestore.net/lowest_popular_accepted_answers_with_unique_tag.parquet'

questions_with_cleaned_text_columns_output_path = f'adl://{datalake_name}.azuredatalakestore.net/questions_with_cleaned_text_columns.parquet'

In [ ]:
%sh

sudo apt-get -y install p7zip-full

wget https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z

7za x stackoverflow.com-Posts.7z

ls -al

In [ ]:
%python

dbutils.fs.mv('file:/databricks/driver/Posts.xml', f'{path}')

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
schema = StructType() \
            .add('_Id', IntegerType()) \
            .add('_PostTypeId', IntegerType()) \
            .add('_AcceptedAnswerId', IntegerType()) \
            .add('_ParentId', IntegerType()) \
            .add('_CreationDate', TimestampType()) \
            .add('_DeletionDate', TimestampType()) \
            .add('_Score', IntegerType()) \
            .add('_ViewCount', IntegerType()) \
            .add('_Body', StringType()) \
            .add('_OwnerUserId', IntegerType()) \
            .add('_Title', StringType()) \
            .add('_Tags', StringType()) \
            .add('_AnswerCount', IntegerType()) \
            .add('_CommentCount', IntegerType()) \
            .add('_FavoriteCount', IntegerType()) \
            .add('_ClosedDate', TimestampType())

In [ ]:
posts_df = spark.read \
      .format('com.databricks.spark.xml') \
      .option('rootTag', 'posts') \
      .option('rowTag', 'row') \
      .load(path, schema = schema)

In [ ]:
posts_df.show()

In [ ]:
posts_df \
        .withColumnRenamed('_Id', 'Id') \
        .withColumnRenamed('_PostTypeId', 'PostTypeId') \
        .withColumnRenamed('_AcceptedAnswerId', 'AcceptedAnswerId') \
        .withColumnRenamed('_ParentId', 'ParentId') \
        .withColumnRenamed('_CreationDate', 'CreationDate') \
        .withColumnRenamed('_DeletionDate', 'DeletionDate') \
        .withColumnRenamed('_Score', 'Score') \
        .withColumnRenamed('_ViewCount', 'ViewCount') \
        .withColumnRenamed('_Body', 'Body') \
        .withColumnRenamed('_OwnerUserId', 'OwnerUserId') \
        .withColumnRenamed('_Title', 'Title') \
        .withColumnRenamed('_Tags', 'Tags') \
        .withColumnRenamed('_AnswerCount', 'AnswerCount') \
        .withColumnRenamed('_CommentCount', 'CommentCount') \
        .withColumnRenamed('_FavoriteCount', 'FavoriteCount') \
        .withColumnRenamed('_ClosedDate', 'ClosedDate') \
        .write \
        .mode('overwrite') \
        .format('parquet') \
        .option('path', posts_output_path) \
        .save()

In [ ]:
posts_based_parquet_df=spark.read.parquet(posts_output_path)
posts_based_parquet_df.show()

In [ ]:
posts_based_parquet_df.count()

In [ ]:
questions_df = posts_based_parquet_df.filter((col('PostTypeId') == 1) & (col('ClosedDate').isNull()))
answers_df = posts_based_parquet_df.filter((col('PostTypeId') == 2) & (col('ClosedDate').isNull()))

## Most popular technologies

In [ ]:
top_10_the_most_popular_technologies_based_on_the_survey_posts = ['<javascript>', '<html>', '<sql>', '<python>', '<typescript>', '<java>', '<c#>', '<bash>', '<php>', '<c++>']

def most_popular_tag_is_in_tags(column_value):
    for tag in top_10_the_most_popular_technologies_based_on_the_survey_posts:
        if tag in column_value:
            return True
    return False

filter_most_popular_tag_is_in_tags = udf(most_popular_tag_is_in_tags, BooleanType())


most_popular_unique_tags_schema = StructType([
    StructField('Tag', StringType())
])
most_popular_unique_tags_data = map(lambda tag: Row(tag), top_10_the_most_popular_technologies_based_on_the_survey_posts)
most_popular_unique_tags_df = spark.createDataFrame(data = most_popular_unique_tags_data, schema = most_popular_unique_tags_schema)

most_popular_questions_with_unique_tag_df = questions_df \
    .filter(filter_most_popular_tag_is_in_tags(questions_df['Tags'])) \
    .crossJoin(most_popular_unique_tags_df) \
    .filter(col('Tags').contains(col('Tag'))) \
    .drop('Tags')


most_popular_questions_with_unique_tag_df \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', most_popular_questions_with_unique_tag_parquet_path) \
    .save()

In [ ]:
most_popular_answers_with_unique_tag_df = answers_df.drop('Tags').alias('a') \
    .join(most_popular_questions_with_unique_tag_df.alias('q'), col('a._ParentId') == col('q._Id')) \
    .select('a.*', 'q.Tag')

most_popular_answers_with_unique_tag_df \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', most_popular_answers_with_unique_tag_parquet_path) \
    .save()

In [ ]:

most_popular_accepted_answers_with_unique_tag_df = answers_df.drop('Tags').alias('a') \
    .join(most_popular_questions_with_unique_tag_df.alias('q'), col('a._Id') == col('q._AcceptedAnswerId')) \
    .select('a.*', 'q.Tag')

most_popular_accepted_answers_with_unique_tag_df \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', most_popular_accepted_answers_with_unique_tag_parquet_path) \
    .save()

## Least popular technologies

In [ ]:
top_10_the_lowest_popular_technologies_based_on_the_survey_posts = ['<crystal-lang>', '<apl>', '<sas>', '<ocaml>', '<cobol>', '<fortran>', '<erlang>', '<julia>', '<f#>', '<lisp>']

def lowest_popular_tag_is_in_tags(column_value):
    for tag in top_10_the_lowest_popular_technologies_based_on_the_survey_posts:
        if tag in column_value:
            return True
    return False

filter_lowest_popular_tag_is_in_tags = udf(lowest_popular_tag_is_in_tags, BooleanType())


lowest_popular_unique_tags_schema = StructType([
    StructField('Tag', StringType())
])
lowest_popular_unique_tags_data = map(lambda tag: Row(tag), top_10_the_lowest_popular_technologies_based_on_the_survey_posts)
lowest_popular_unique_tags_df = spark.createDataFrame(data = lowest_popular_unique_tags_data, schema = lowest_popular_unique_tags_schema)

lowest_popular_questions_with_unique_tag_df = questions_df \
    .filter(filter_lowest_popular_tag_is_in_tags(questions_df['Tags'])) \
    .crossJoin(lowest_popular_unique_tags_df) \
    .filter(col('Tags').contains(col('Tag'))) \
    .drop('Tags')

lowest_popular_questions_with_unique_tag_df \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', lowest_popular_questions_with_unique_tag_parquet_path) \
    .save()

In [ ]:
lowest_popular_answers_with_unique_tag_df = answers_df.drop('Tags').alias('a') \
    .join(lowest_popular_questions_with_unique_tag_df.alias('q'), col('a._ParentId') == col('q._Id')) \
    .select('a.*', 'q.Tag')

lowest_popular_answers_with_unique_tag_df \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', lowest_popular_answers_with_unique_tag_parquet_path) \
    .save()

In [ ]:
lowest_popular_accepted_answers_with_unique_tag_df = answers_df.drop('Tags').alias('a') \
    .join(lowest_popular_questions_with_unique_tag_df.alias('q'), col('a._Id') == col('q._AcceptedAnswerId')) \
    .select('a.*', 'q.Tag')

lowest_popular_accepted_answers_with_unique_tag_df \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', lowest_popular_accepted_answers_with_unique_tag_parquet_path) \
    .save()

## Preparing text columns Title and Body - with setting up NLTK and Environment Variable

In [ ]:
dbutils.fs.put('dbfs:/databricks/scripts/nltk-install.sh', """
#!/bin/bash
pip install nltk""", True)

In [ ]:
display(dbutils.fs.ls('dbfs:/databricks/scripts/'))

In [ ]:
%sh
head '/dbfs/databricks/scripts/nltk-install.sh'

In [ ]:
#SET Environment Variable on Azure Databricks -> NLTK_DATA=/dbfs/databricks/nltk_data/
import os
os.environ.get('NLTK_DATA')

In [ ]:
#Verification that NLTK is properly installed
import nltk
nltk.data.path

In [ ]:
import nltk as nltk
from nltk.corpus import stopwords
nltk.download('stopwords', download_dir='/dbfs/databricks/nltk_data/')
from nltk.tokenize import word_tokenize
nltk.download('punkt', download_dir='/dbfs/databricks/nltk_data/')
from nltk.stem.snowball import SnowballStemmer

In [ ]:
questions_without_duplicates = questions_df \
.filter((col('Title').isNotNull()) & (col('Body').isNotNull()) & (col('Tags').isNotNull())) \
.dropDuplicates(['Title', 'Body', 'Tags'])

In [ ]:
def remove_html(text):
    text.encode('utf-8')
    cleaner = re.compile('<.*?>')
    clean_text = re.sub(cleaner, ' ', str(text))
    return clean_text

remove_html_from_text_udf = udf(remove_html, StringType())

In [ ]:
questions_without_html = questions_without_duplicates \
    .withColumn('Body', remove_html_from_text_udf(questions_without_duplicates['Body']))

questions_without_html.show()

In [ ]:
def remove_special_characters(text):
    text.encode('utf-8')
    cleaner = re.compile('[^A-Za-z0-9#+.\-]+')
    clean_text = re.sub(cleaner, ' ', str(text))
    return clean_text

remove_special_characters_udf = udf(remove_special_characters, StringType())

In [ ]:
questions_without_html_and_special = questions_without_html \
    .withColumn('Body', remove_special_characters_udf(questions_without_html['Body'])) \
    .withColumn('Title', remove_special_characters_udf(questions_without_html['Title']))

questions_without_html_and_special.show()

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def remove_stop_words_without_c(text):
    text.encode('utf-8')
    words=word_tokenize(str(text.lower()))
    return ' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))



remove_stop_words_without_c_udf = udf(remove_stop_words_without_c, StringType())

In [ ]:
questions_with_cleaned_text_columns = questions_without_html_and_special \
    .withColumn('Body', remove_stop_words_without_c_udf(questions_without_html_and_special['Body'])) \
    .withColumn('Title', remove_stop_words_without_c_udf(questions_without_html_and_special['Title']))

questions_with_cleaned_text_columns.show()

In [ ]:
questions_with_cleaned_text_columns \
    .write \
    .mode('overwrite') \
    .format('parquet') \
    .option('path', questions_with_cleaned_text_columns_output_path) \
    .save()